### Generating the QR

In [2]:
import qrcode
qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=1,
        border=1,
    )
    
qr.add_data("8j#hgj*8")

qr.make(fit=True)

img = qr.make_image(fill_color="black", back_color="white")
img1 = img.convert("RGB")

img1.save('resources/output/inner_qr.png')

img1.show() 

In [3]:
from PIL import Image
image = Image.open('resources/output/inner_qr.png')
enc_array = list(image.getdata())

### Shuffling the QR

In [4]:
import random
def shuffle_under_seed(ls, seed):
    random.seed(seed)
    random.shuffle(ls)
    return ls

In [6]:
original_list = enc_array
seed = 42
shuffled = shuffle_under_seed(original_list.copy(), seed)

In [7]:
from PIL import Image

image = Image.new("RGB", (23, 23))
image.putdata(shuffled)
image.show()
image.save("resources/output/shuffled_inner_qr.png")

In [8]:
image_1 = Image.open('resources/output/shuffled_inner_qr.png')

In [9]:
enc_array_1 = list(image_1.getdata())

In [10]:
def unshuffle_list(shuffled_ls, seed):
    n = len(shuffled_ls)
    indices = [i for i in range(1, n + 1)]
    shuffled_indices = shuffle_under_seed(indices, seed)
    zipped_ls = list(zip(shuffled_ls, shuffled_indices))
    zipped_ls.sort(key=lambda x: x[1])
    return [a for (a, b) in zipped_ls]

### Forming the formula to Deblur

In [11]:
from aes_cipher import DataEncrypter, DataDecrypter
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES

def adjust_password(password):
    # Truncate or pad the password to 32 bytes (256 bits)
    if len(password) > 32:
        return password[:32]
    elif len(password) < 32:
        return password.ljust(32, b'\x00')  # Padding with null bytes
    return password

def encrypt_AES_ECB(message, password):
    password = adjust_password(password)
    cipher = AES.new(password, AES.MODE_ECB)
    padded_message = pad(message, AES.block_size)
    encrypted_data = cipher.encrypt(padded_message)
    return encrypted_data

def decrypt_AES_ECB(encrypted_data, password):
    password = adjust_password(password)
    cipher = AES.new(password, AES.MODE_ECB)
    decrypted_data = cipher.decrypt(encrypted_data)
    unpadded_data = unpad(decrypted_data, AES.block_size)
    return unpadded_data

password = b"supersecretpassword"
message = str(seed).encode()

encrypted_message_ECB = encrypt_AES_ECB(message, password)

decrypted_message_ECB = decrypt_AES_ECB(encrypted_message_ECB, password)

print("Original message:", message)
print("Encrypted message (ECB):", encrypted_message_ECB)
print("Decrypted message (ECB):", decrypted_message_ECB)

Original message: b'42'
Encrypted message (ECB): b'\x18\xa4B\xcc\xb6\x18\x0fv$\xa5\xde+\r\xa1\x97['
Decrypted message (ECB): b'42'


### Add border

In [15]:
im = Image.open("resources/output/shuffled_inner_qr.png")

border = Image.open('resources/border.png')
pos = ((border.size[0] - im.size[0]) // 2, (border.size[1] - im.size[1]) // 2)

border.paste(im, pos)
border.show()
border.save('resources/output/border_with_inner_qr_1.png')

### Encoding the Formula into the outer QR

In [16]:
# Encrypt the message
encrypted_message = encrypt_AES_ECB(message, password)
# Decrypt the message
decrypted_message = decrypt_AES_ECB(encrypted_message, password)

In [17]:
qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_H,
        box_size=3,
        border=1,
    )
    
qr.add_data(str(encrypt_AES_ECB(message, password))[2:-1])
# qr.add_data(result)
# qr.make(fit=True)

# print(enc)
img = qr.make_image(fill_color="black", back_color="white").convert("RGB")
#img1 = img.convert("RGB")
img = img.resize((81,81))
img.save('resources/output/outer_qr.png')

img.show()

### Add Inner QR with border to Outer QR

In [21]:
im = Image.open("resources/output/border_with_inner_qr_1.png").convert('L')

border = Image.open('resources/output/outer_qr.png').convert('L')

pos = ((border.size[0] - im.size[0]) // 2, (border.size[1] - im.size[1]) // 2)

border.paste(im, pos)
border.save('resources/output/outer_with_inner_qr_1.png')
border.show()


### Reading the outer QR

In [22]:
from qreader import QReader
import cv2

# Create a QReader instance
qreader = QReader()

# Get the image that contains the QR code
image = cv2.cvtColor(cv2.imread("resources/output/outer_with_inner_qr_1.png"), cv2.COLOR_BGR2RGB)

# Use the detect_and_decode function to get the decoded QR data
decoded_text = qreader.detect_and_decode(image=image)
decoded_text[0]

In [ ]:
try:
    b64 = json.loads(temp)
    nonce = b64decode(b64['nonce'])
    ct = b64decode(b64['ciphertext'])
    cipher = AES.new(key, AES.MODE_CTR, nonce=nonce)
    pt = cipher.decrypt(ct)
    print("The message was: ", pt)
except (ValueError, KeyError):
    print("Incorrect decryption")

The message was:  b'L\x96'


In [ ]:
from PIL import Image
from pyzbar import pyzbar

# img = Image.open('outer_with_inner_qr_1.png')
img = Image.open("resources/output/outer_with_inner_qr_1.png")
output = pyzbar.decode(img)
print(output)
#text = str(output[0][0])
str(output[0][0])[2:-1]
# Replace "\\" with "\"
# text = text.replace('\\\\', '\\')
#text
# print(text)

In [ ]:
try:
    b64 = json.loads(str(output[0][0])[2:-1])
    nonce = b64decode(b64['nonce'])
    ct = b64decode(b64['ciphertext'])
    cipher = AES.new(key, AES.MODE_CTR, nonce=nonce)
    pt = cipher.decrypt(ct)
    print("The message was: ", pt)
except (ValueError, KeyError):
    print("Incorrect decryption")

The message was:  b'L\x96'


### Decoding the Encoded Formula

In [24]:
decrypted_message_ECB = decrypt_AES_ECB(encrypted_message_ECB, password)
print("Decrypted message (ECB):", decrypted_message_ECB)

Decrypted message (ECB): b'42'


In [25]:
seed = int(str(decrypted_message_ECB)[2:-1])
seed

42

### Cropping and Scaling the inner QR

In [26]:
from PIL import Image

def crop_and_scale_image(image_path, top_pixels, bottom_pixels, left_pixels, right_pixels, scale_factor):
    
    image = Image.open(image_path)
    
    width, height = image.size
    
    cropped_image = image.crop((left_pixels, top_pixels, width - right_pixels, height - bottom_pixels))
    
    # new_width = int((width - left_pixels - right_pixels) * scale_factor)
    # new_height = int((height - top_pixels - bottom_pixels) * scale_factor)
    
    # scaled_image = cropped_image.resize((new_width, new_height))
    
    # return scaled_image
    return cropped_image

image_path = 'resources/output/outer_with_inner_qr_1.png'
top_pixels = 29
bottom_pixels = 29
left_pixels = 29
right_pixels = 29
scale_factor = 5

scaled_image = crop_and_scale_image(image_path, top_pixels, bottom_pixels, left_pixels, right_pixels, scale_factor)

scaled_image.show()
scaled_image.save('resources/output/scaled_image.png')

### Using the formula to decode the inner QR

In [30]:
image_temp = Image.open('resources/output/scaled_image.png').convert('RGB')
enc_array_temp = list(image_temp.getdata())
unshuffled = unshuffle_list(enc_array_temp, 42)

In [31]:
from PIL import Image

# Create a new image with RGB mode
image = Image.new("RGB", (23, 23))

# Set pixel values for the image
image.putdata(unshuffled)

# Save or display the image
image.show()  # Display the image
image.save("resources/output/final_output.png")  # Save the image to a file

### Reading the inner QR

In [32]:
img_temp = Image.open('resources/output/final_output.png')
output_temp = pyzbar.decode(img_temp)
text_temp = str(output_temp[0][0])
print(text_temp[2:-1])

8j#hgj*8


### Detecting, Scanning and Processing the Physical QR

In [ ]:
import cv2
from qrdet import QRDetector

def crop_ROI(img):
    detector = QRDetector()
    # image = cv2.imread(img)
    detections = detector.detect(image=img, is_bgr=True)
    if detections :
        
        g= detections[0]
        quad= g['quad_xy']
        dec=g['bbox_xyxy']

        x= int(dec[0])
        y= int(dec[1])
        w= int(dec[2])
        h= int(dec[3])
        
        cropped_img = img[y:h, x:w]
        cv2.imshow('cropped', cropped_img)
        return cropped_img
    return None

In [ ]:
vid = cv2.VideoCapture('http://192.168.70.30:8080/video')
# 
while(True): 
    ret, frame = vid.read() 
    frame=frame[500:1000,500:1000]
    cv2.imshow('FRAME',frame)
    k=cv2.waitKey(1) & 0xFF
    if k== ord('y'): 
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # thresh = 125
        # im_bw = cv2.threshold(gray, thresh, 255, cv2.THRESH_BINARY)[1]
        (thresh, im_bw) = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        print(thresh)
        img= crop_ROI(im_bw)
        cv2.imwrite('resources/captured/captured_qr.png',img)
        if k== ord('a'): 
            break

    elif k == ord("q"):
        break

cv2.destroyAllWindows() 
vid.release()

In [ ]:
from PIL import Image

def image_to_binary_image(image_path, threshold=100):
  img = Image.open(image_path).convert('L')  # Convert to grayscale
  binary_img = img.point(lambda p: 0 if p < threshold else 255)
  return binary_img

# Example usage
binary_image = image_to_binary_image("resources/captured/resized_image.png", threshold= 90)
binary_image.save("resources/captured/binary_image.png")
binary_image.show()

In [ ]:
img = Image.open("resources/captured/Captured_TruQR.png").convert('RGB')
resized_img = img.resize((79, 79), Image.LANCZOS)
resized_img.save("resources/captured/resized_image.jpg")
resized_img.show()


### Check if TruQR has inner QR by detecting border

In [ ]:
import cv2
import numpy as np

# Read the image
image = cv2.imread('resources/output/qr.png')

# Convert the image to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# cv2.imshow('gray image',gray_image)
# Apply GaussianBlur to reduce noise
blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
# cv2.imshow('blurred_image', blurred_image)
# Perform edge detection using Canny
canny_edges = cv2.Canny(blurred_image, 50, 150)
# cv2.imshow('canny image',canny_edges)
# Find contours
contours, _ = cv2.findContours(canny_edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Define minimum contour area
min_contour_area = 1000

# Draw contours on the original image if contour area is greater than minimum
for contour in contours:
    contour_area = cv2.contourArea(contour)
    if contour_area > min_contour_area:
        cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
        x,y,w,h = cv2.boundingRect(contour)
        cv2.putText(image, str(w), (x,y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 1)

cropped_img = image[y:y+h, x:x+w]

# Display the result
cv2.imshow('Contours', cropped_img)
cv2.imwrite('resources/output/contour.png', cropped_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Cryptographic Methods

In [ ]:
import cryptocode
myEncryptedMessage = cryptocode.encrypt("42", "password123")
print(myEncryptedMessage)
import cryptocode
myDecryptedMessage = cryptocode.decrypt("hrs=*AzYACoTnid5plcGg6zGvcw==*36obinSZpBx3jMG84sO0sw==*H2DUi3Vx5Gj07mqxwWFXSQ==", "password123")
print(myDecryptedMessage)
key = get_random_bytes(16)
def enc(data, key):
    data = data.encode()

    cipher = AES.new(key, AES.MODE_CFB)
    ct_bytes = cipher.encrypt(data)
    iv = b64encode(cipher.iv).decode('utf-8')
    ct = b64encode(ct_bytes).decode('utf-8')
    print('byte arr: ', ct_bytes, '\nini vec: ',iv, '\nciphertext: ', ct)
    print()
    return iv, ct
def decrypt(iv, ct, key):
    try:
        iv = b64decode(iv)
        ct = b64decode(ct)
        cipher = AES.new(key, AES.MODE_CFB, iv=iv)
        pt = cipher.decrypt(ct)
        print("The message was: ", pt)
    except (ValueError, KeyError):
        print("Incorrect decryption")
n,c = enc('idk wht happens', key)
decrypt(n,c,key)
import json
from base64 import b64encode
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

data = b"12543"
key = b"secret password "
cipher = AES.new(key, AES.MODE_CTR)
ct_bytes = cipher.encrypt(data)
nonce = b64encode(cipher.nonce).decode('utf-8')
ct = b64encode(ct_bytes).decode('utf-8')
result = json.dumps({'nonce':nonce, 'ciphertext':ct})
print(result)
import json
from base64 import b64decode
from Crypto.Cipher import AES

# We assume that the key was securely shared beforehand
try:
    b64 = json.loads(result)
    nonce = b64decode(b64['nonce'])
    ct = b64decode(b64['ciphertext'])
    cipher = AES.new(key, AES.MODE_CTR, nonce=nonce)
    pt = cipher.decrypt(ct)
    print("The message was: ", pt)
except (ValueError, KeyError):
    print("Incorrect decryption")